# Notebook #2: Data Preprocessing and Cleaning

In [172]:
%load_ext sql
%config SqlMagic.autocommit=True 
%sql sqlite:///../data/Spotifyartist.db --alias spotifyartist

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [173]:
import os
import json

import pandas as pd

import sqlite3

# 2.1 Extract Artist Info and Create Pandas Dataframe 

In [196]:

# Directory containing JSON files
directory = '../data/raw'

# List to hold data from each JSON file
data_list = []

def extract_artist_info(artist):
    return {
        "id": artist["id"],
        "name": artist["name"],
        "followers_total": artist["followers"]["total"],
        "genres": ", ".join(artist["genres"]),
        "popularity": artist["popularity"],
        "spotify_url": artist["external_urls"]["spotify"],
        "image_url": artist["images"][0]["url"] if artist["images"] else None,
        "uri": artist["uri"]
    }

# Iterate over all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.json'):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r') as file:
            data = json.load(file)
            artist_info = extract_artist_info(data)
            data_list.append(artist_info)

# Create a DataFrame from the list of extracted artist information
df = pd.DataFrame(data_list)

df.head()

,id,name,followers_total,genres,popularity,spotify_url,image_url,uri
0,3444S3C4U9Ts86BnCtSPRV,Treaty Oak Revival,232325,"modern country pop, texas country",71,https://open.spotify.com/artist/3444S3C4U9Ts86...,https://i.scdn.co/image/ab6761610000e5eb4c287c...,spotify:artist:3444S3C4U9Ts86BnCtSPRV
1,4YLtscXsxbVgi031ovDDdh,Chris Stapleton,7245108,"contemporary country, outlaw country",82,https://open.spotify.com/artist/4YLtscXsxbVgi0...,https://i.scdn.co/image/ab6761610000e5eba0c87c...,spotify:artist:4YLtscXsxbVgi031ovDDdh
2,6x2LnllRG5uGarZMsD4iO8,Thomas Rhett,4708139,"contemporary country, country, country road, m...",74,https://open.spotify.com/artist/6x2LnllRG5uGar...,https://i.scdn.co/image/ab6761610000e5eb4aae48...,spotify:artist:6x2LnllRG5uGarZMsD4iO8
3,1Tie3AZgLQZqYEp8Fv4zOZ,Koe Wetzel,757205,"classic texas country, modern country pop",75,https://open.spotify.com/artist/1Tie3AZgLQZqYE...,https://i.scdn.co/image/ab6761610000e5ebef6319...,spotify:artist:1Tie3AZgLQZqYEp8Fv4zOZ
4,6y8XlgIV8BLlIg1tT1R10i,Old Dominion,1543299,"contemporary country, country, country road, m...",70,https://open.spotify.com/artist/6y8XlgIV8BLlIg...,https://i.scdn.co/image/ab6761610000e5ebd97f87...,spotify:artist:6y8XlgIV8BLlIg1tT1R10i


# 2.2 Create SQL .db file to store all the data

In [237]:
artists_df = df[['id', 'name', 'followers_total', 'popularity']]

# Create a DataFrame for genres table with id, genre, followers, and spotify_url
genres_df = df.explode('genres')[['id', 'genres', 'followers_total', 'spotify_url', 'name']]

# Save the data to a SQLite database
data_folder = '../data/clean'
db_path = os.path.join(data_folder, 'Spotifyartist.db')
conn = sqlite3.connect(db_path)

artists_df.to_sql('artists', conn, if_exists='replace', index=False)
genres_df.to_sql('genres', conn, if_exists='replace', index=False)

# 2.3 Data Tables using sqlite3

In [238]:
#Create table 1
pd.read_sql(
    '''
    SELECT *
    FROM artists
    LIMIT 50
    ''', conn
)


,id,name,followers_total,popularity
0,3444S3C4U9Ts86BnCtSPRV,Treaty Oak Revival,232325,71
1,4YLtscXsxbVgi031ovDDdh,Chris Stapleton,7245108,82
2,6x2LnllRG5uGarZMsD4iO8,Thomas Rhett,4708139,74
3,1Tie3AZgLQZqYEp8Fv4zOZ,Koe Wetzel,757205,75
4,6y8XlgIV8BLlIg1tT1R10i,Old Dominion,1543299,70
5,4oUHIQIBe0LHzYfvXNW4QM,Morgan Wallen,10040104,91
6,4mxWe1mtYIYfP040G38yvS,Alan Jackson,2759900,73
7,19k8AgwwTSxeaxkOuCQEJs,Jelly Roll,3894300,79
8,70kkdajctXSbqSMJbQO424,Kacey Musgraves,1441136,76
9,5Ppie0uPnbnvGBYRwYmlt0,Megan Moroney,406030,74


In [211]:
#Create table 2

pd.read_sql(
    '''
    SELECT *
    FROM genres
    LIMIT 50
    ''', conn
)


,id,genre,followers_total,spotify_url
0,3444S3C4U9Ts86BnCtSPRV,"modern country pop, texas country",232325,https://open.spotify.com/artist/3444S3C4U9Ts86...
1,4YLtscXsxbVgi031ovDDdh,"contemporary country, outlaw country",7245108,https://open.spotify.com/artist/4YLtscXsxbVgi0...
2,6x2LnllRG5uGarZMsD4iO8,"contemporary country, country, country road, m...",4708139,https://open.spotify.com/artist/6x2LnllRG5uGar...
3,1Tie3AZgLQZqYEp8Fv4zOZ,"classic texas country, modern country pop",757205,https://open.spotify.com/artist/1Tie3AZgLQZqYE...
4,6y8XlgIV8BLlIg1tT1R10i,"contemporary country, country, country road, m...",1543299,https://open.spotify.com/artist/6y8XlgIV8BLlIg...
5,4oUHIQIBe0LHzYfvXNW4QM,contemporary country,10040104,https://open.spotify.com/artist/4oUHIQIBe0LHzY...
6,4mxWe1mtYIYfP040G38yvS,country,2759900,https://open.spotify.com/artist/4mxWe1mtYIYfP0...
7,19k8AgwwTSxeaxkOuCQEJs,"country rap, nashville hip hop, tennessee hip hop",3894300,https://open.spotify.com/artist/19k8AgwwTSxeax...
8,70kkdajctXSbqSMJbQO424,"classic texas country, country dawn",1441136,https://open.spotify.com/artist/70kkdajctXSbqS...
9,5Ppie0uPnbnvGBYRwYmlt0,"contemporary country, modern country pop",406030,https://open.spotify.com/artist/5Ppie0uPnbnvGB...


In [234]:

# Create Table 3 by using data from tables 1 and 2

pd.read_sql(
   '''
   SELECT
       a.*,
       GROUP_CONCAT(g.genres) AS genres
   FROM
       artists a
   LEFT JOIN
       genres g
   ON
       a.name = g.name AND a.popularity = a.popularity
   GROUP BY
       a.name
   LIMIT 50
   ''', conn
)


,id,name,followers_total,popularity,genres
0,4mxWe1mtYIYfP040G38yvS,Alan Jackson,2759900,73,country
1,3win9vGIxFfBRag9S63wwf,Bailey Zimmerman,1243448,75,"contemporary country, modern country pop"
2,1UTPBmNbXNTittyMJrNkvw,Blake Shelton,7172304,74,"classic oklahoma country, contemporary country..."
3,0XKOBt59crntr7HQXXO8Yz,Brooks & Dunn,2910162,71,"contemporary country, country, country road"
4,0ChjBYedhZTQnWZWQYg15U,Bryan Martin,172352,70,modern country pop
5,4xFUf1FHVy696Q1JQZMTRj,Carrie Underwood,5958908,69,"classic oklahoma country, contemporary country..."
6,4YLtscXsxbVgi031ovDDdh,Chris Stapleton,7245108,82,"contemporary country, outlaw country"
7,6zLBxLdl60ekBLpawtT63I,Cody Johnson,1363299,75,"classic texas country, contemporary country, m..."
8,3xYXYzm9H3RzyQgBrYwIcx,Colter Wall,1326523,69,"canadian americana, canadian contemporary coun..."
9,3IYUhFvPQItj6xySrBmZkd,Creedence Clearwater Revival,6726341,81,"album rock, classic rock, country rock, mellow..."
